In [141]:
import math
import numpy as np
import gdal
from copy import deepcopy


In [142]:
def readraster(file):
    dataSource = gdal.Open(file)
    band = dataSource.GetRasterBand(1)
    band = band.ReadAsArray()
    return (dataSource, band)


In [143]:
def identicalList(inList):
    global logical
    inList = np.array(inList)
    logical = inList == inList[0]
    if sum(logical) == len(inList):
        return True
    else:
        return False


In [207]:
def builtupAreaDifference(landcover1, landcover2, buclass=1, cellsize=30):
    return (sum(sum(((landcover2 == buclass).astype(int) - (landcover1 == buclass).astype(int) != 0)))
            * (cellsize**2) / 1000000)


In [208]:
class landcover:
    def __init__(self, file1, file2):
        self.ds_lc1, self.arr_lc1 = readraster(file1)
        self.ds_lc2, self.arr_lc2 = readraster(file2)
        self.performChecks()

    def performChecks(self):
        # check the rows and columns of input land cover datasets
        print("Checking the size of input rasters...")
        if (self.ds_lc1.RasterXSize == self.ds_lc2.RasterXSize) and (self.ds_lc1.RasterYSize == self.ds_lc2.RasterYSize):
            print("Land cover data size matched.")
            self.row, self.col = (self.ds_lc1.RasterYSize, self.ds_lc1.RasterXSize)
        else:
            print("Input land cover files have different height and width.")

        # Check the number of classes in input land cover images
        print("\nChecking feature classes in land cover data...")
        if (self.arr_lc1.max() == self.arr_lc2.max()) and (self.arr_lc1.min() == self.arr_lc2.min()):
            print("The classes in input land cover files are matched.")
            self.nFeature = self.arr_lc1.max() - self.arr_lc1.min() + 1
            # print("HIIIIIIII : nFeature = ", self.nFeature)
        else:
            print("Input land cover data have different class values/ size.")

    def transitionMatrix(self):
        self.tMatrix = np.random.randint(1, size=(self.nFeature, self.nFeature))
        for x in range(0, self.row):
            for y in range(0, self.col):
                t1_pixel = self.arr_lc1[x, y]
                t2_pixel = self.arr_lc2[x, y]
                self.tMatrix[t1_pixel - 1, t2_pixel - 1] += 1
        self.tMatrixNorm = np.random.randint(1, size=(4, 5)).astype(float)
        print("\nTransition Matrix computed, normalisation in progress..")
        # Creating normalised transition matrix
        for x in range(0, self.tMatrix.shape[0]):
            for y in range(0, self.tMatrix.shape[1]):
                self.tMatrixNorm[x, y] = self.tMatrix[x, y] / (self.tMatrix[x, :]).sum()


In [209]:
class growthFactors:
    def __init__(self, *args):
        self.gf = dict()
        self.gf_ds = dict()
        self.nFactors = len(args)
        n = 1
        for file in args:
            self.gf_ds[n], self.gf[n] = readraster(file)
            n += 1
        self.performChecks()

    def performChecks(self):
        print("\nChecking the size of input growth factors...")
        rows = []
        cols = []
        for n in range(1, self.nFactors + 1):
            rows.append(self.gf_ds[n].RasterYSize)
            cols.append(self.gf_ds[n].RasterXSize)
        if (identicalList(rows) == True) and ((identicalList(cols) == True)):
            print("Input factors have same row and column value.")
            self.row = self.gf_ds[n].RasterYSize
            self.col = self.gf_ds[n].RasterXSize
        else:
            print("Input factors have different row and column value.")


In [210]:
class fitmodel:
    def __init__(self, landcoverClass, growthfactorsClass, kernelSize = 3):
        self.landcovers = landcoverClass
        self.factors = growthfactorsClass
        self.performChecks()
        self.kernelSize = kernelSize

    def performChecks(self):
        print("\nMatching the size of land cover and growth factors...")
        if (self.landcovers.row == self.factors.row) and (self.factors.col == self.factors.col):
            print("Size of rasters matched.")
            self.row = self.factors.row
            self.col = self.factors.col
        else:
            print("ERROR! Raster size not matched please check.")

    def setThreshold(self, builtupThreshold, *OtherThresholdsInSequence):
        self.threshold = list(OtherThresholdsInSequence)
        self.builtupThreshold = builtupThreshold
        if len(self.threshold) == (len(self.factors.gf)):
            print("\nThreshold set for factors")
        else:
            print("ERROR! Please check the number of factors.")

    def predict(self):
        self.predicted = deepcopy(self.landcovers.arr_lc1)
        sideMargin = math.floor(self.kernelSize / 2)
        for x in range(sideMargin, self.row - sideMargin):
            for y in range(sideMargin, self.col - sideMargin):
                kernel = self.landcovers.arr_lc1[x - sideMargin : x + sideMargin + 1, y - sideMargin : y + sideMargin + 1]
                builtupCount = sum(sum(kernel == 1))
                # If the number of built-up cells greater than equal to assigned threshold
                # if (builtupCount >= self.builtupThreshold):
                if (builtupCount >= self.builtupThreshold) and (self.factors.gf[3][x, y] != 1):  # Adding exception for the restricted areas
                    for factor in range(1, self.factors.nFactors + 1):
                        # If the assigned thresholds are less than zero, then the smaller than rule applies, else greater than
                        if self.threshold[factor - 1] < 0:
                            if self.factors.gf[factor][x, y] <= abs(self.threshold[factor - 1]):
                                self.predicted[x, y] = 1
                                break
                            else:
                                pass
                        elif self.threshold[factor - 1] > 0:
                            if (self.factors.gf[factor][x, y] >= self.threshold[factor - 1]):
                                self.predicted[x, y] = 1
                                break
                            else:
                                pass
                if (x % 500 == 0) and (y % 500 == 0):
                    print("Row: %d, Col: %d, Builtup cells count: %d\n" % (x, y, builtupCount), end="\r", flush=True, )

    def checkAccuracy(self):
        # Statistical Accuracy
        self.actualBuildup = builtupAreaDifference(self.landcovers.arr_lc1, self.landcovers.arr_lc2)
        self.predictedBuildup = builtupAreaDifference(self.landcovers.arr_lc1, self.predicted)
        self.spatialAccuracy = 100 - (sum(sum(( (self.predicted == 1).astype(float) - (self.landcovers.arr_lc2 == 1).astype(float) ) != 0)) / sum(sum(self.landcovers.arr_lc2 == 1))) * 100
        
        print("Actual growth: %d, Predicted growth: %d" % (self.actualBuildup, self.predictedBuildup))
        # Spatial Accuracy
        print("Spatial accuracy: %f" % (self.spatialAccuracy))

    def exportPredicted(self, outFileName):
        driver = gdal.GetDriverByName("GTiff")
        outdata = driver.Create(outFileName, self.col, self.row, 1, gdal.GDT_UInt16)  # option: GDT_UInt16, GDT_Float32
        outdata.SetGeoTransform(self.landcovers.ds_lc1.GetGeoTransform())
        outdata.SetProjection(self.landcovers.ds_lc1.GetProjection())
        outdata.GetRasterBand(1).WriteArray(self.predicted)
        outdata.GetRasterBand(1).SetNoDataValue(0)
        outdata.FlushCache()
        outdata = None


In [211]:
# Input land cover GeoTIFF for two time period
# file1 = "LULC_2014.tif"
# file2 = "LULC_2015.tif"

file1 = ""
file2 = ""

LULC2014 = "LULC/LULC_2014.tif"
LULC2015 = "LULC/LULC_2015.tif"
LULC2016 = "LULC/LULC_2016.tif"
LULC2017 = "LULC/LULC_2017.tif"
LULC2018 = "LULC/LULC_2018.tif"
LULC2019 = "LULC/LULC_2019.tif"
LULC2020 = "LULC/LULC_2020.tif"
LULC2021 = "LULC/LULC_2021.tif"
LULC2022 = "LULC/LULC_2022.tif"

# Input all the parameters
pop2014 = "PopulationDensity/PD_2014.tif"
pop2015 = "PopulationDensity/PD_2015.tif"
pop2016 = "PopulationDensity/PD_2016.tif"
pop2017 = "PopulationDensity/PD_2017.tif"
pop2018 = "PopulationDensity/PD_2018.tif"
pop2019 = "PopulationDensity/PD_2019.tif"
pop2020 = "PopulationDensity/PD_2020.tif"
pop2021 = "PopulationDensity/PD_2021.tif"

slope = "slopeMap.tif"

road = "primary_proximity.tif"

water2014 = "WaterBodies/Water_2014.tif"
water2015 = "WaterBodies/Water_2015.tif"
water2016 = "WaterBodies/Water_2016.tif"
water2017 = "WaterBodies/Water_2017.tif"
water2018 = "WaterBodies/Water_2018.tif"
water2019 = "WaterBodies/Water_2019.tif"
water2020 = "WaterBodies/Water_2020.tif"
water2021 = "WaterBodies/Water_2021.tif"
water2022 = "WaterBodies/Water_2022.tif"
# cbd = "cbddist.tif"


In [213]:
y1 = 2021
y2 = 2022

LULC1 = 'file1'
LULC2 = 'file2'
pop = 'pop'
water = 'water'

exec(LULC1 + f" = LULC{y1}")
exec(LULC2 + f" = LULC{y2}")

exec(pop + f" = pop{y1}")

exec(water + f" = water{y1}")

print(file1)
print(file2)
print(pop)
print(water)

LULC/LULC_2021.tif
LULC/LULC_2022.tif
PopulationDensity/PD_2021.tif
WaterBodies/Water_2021.tif


In [214]:
# Create a land cover class which takes land cover data for two time period
myLandcover = landcover(file1, file2)

# Create a factors class that configures all the factors for the model
# If raster layer showing restricted areas is not involved, kindly uncomment line 157, and comment-out line 158
myFactors = growthFactors(pop, slope, road, water)
# myFactors = growthFactors(cbd, road, pop01, slope)




Checking the size of input rasters...
Land cover data size matched.

Checking feature classes in land cover data...
The classes in input land cover files are matched.

Checking the size of input growth factors...
Input factors have same row and column value.


In [215]:
# Initiate the model with the above created class
caModel = fitmodel(myLandcover, myFactors, 3)


Matching the size of land cover and growth factors...
Size of rasters matched.


In [216]:
# Set the threshold values, Assign negative threshold values if less than rule is required
# Based on the statistical and spatial accuracy displayed, the thresholds should be tweaked
# caModel.setThreshold(2, 10000, -15, -50, -0.5)
caModel.setThreshold(4, 30000, -15, -50, -0.5)
# caModel.setThreshold(3, 1000, 10, -200, -0.5) # Best
                 # (BUT, pop, slope, road, water)
# caModel.setThreshold(3, -15000, -10000, 8000, -3)


Threshold set for factors


In [217]:
# Run the model
caModel.predict()

Row: 500, Col: 500, Builtup cells count: 0
Row: 500, Col: 1000, Builtup cells count: 5
Row: 500, Col: 1500, Builtup cells count: 0
Row: 500, Col: 2000, Builtup cells count: 0
Row: 1000, Col: 500, Builtup cells count: 0
Row: 1000, Col: 1000, Builtup cells count: 6
Row: 1000, Col: 1500, Builtup cells count: 6
Row: 1000, Col: 2000, Builtup cells count: 0
Row: 1500, Col: 500, Builtup cells count: 0
Row: 1500, Col: 1000, Builtup cells count: 5
Row: 1500, Col: 1500, Builtup cells count: 0
Row: 1500, Col: 2000, Builtup cells count: 0
Row: 2000, Col: 500, Builtup cells count: 0
Row: 2000, Col: 1000, Builtup cells count: 0
Row: 2000, Col: 1500, Builtup cells count: 0
Row: 2000, Col: 2000, Builtup cells count: 0


In [218]:
# Check the accuracy of the predicted values
caModel.checkAccuracy()

# Export the predicted layer
# caModel.exportPredicted("181126_predictedlandcover_ra_30m_utm43n_PT_5.tif")
caModel.exportPredicted(f"Predicted_LULC_for_{y2}.tif")


Actual growth: 322, Predicted growth: 108
Spatial accuracy: 64.034441
